In [20]:
import pandas as pd
from dotenv import load_dotenv
import os
import google.generativeai as genai
import json
import time
import sqlite3
from concurrent.futures import ThreadPoolExecutor, as_completed
%load_ext autoreload
%autoreload 2
from gemini_prompts import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Model Config
- You need a .env file with GEMINI_API_KEY defined

In [21]:
load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY_PAID"))

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-pro-002",
  generation_config=generation_config,
)


Check that model works

In [ ]:
# response = model.generate_content("Hello there, what is your name ")
# print(response.text)

ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).

## Function to form prompts

In [23]:
context =  context_local_dataset_v1 + context_single_field_v1
answer_format = answer_format_v1
field_to_query = field_to_query_v1

In [24]:
def form_prompt(query, local_data):
    prompt = f"""

    {context}
    {query}
    {answer_format}

    Here are the results of my search for this firm.
    {local_data}
    """
    return prompt

### Open firm databases

In [25]:
conn = sqlite3.connect("firm_database_llm.db")
cursor = conn.cursor()


# Drop the table if you're starting from scratch
# cursor.execute('''
# DROP TABLE IF EXISTS firm_properties
#                ''')

table_name = "firm_properties_gemini_without_grounding_local_dataset_v1"
# You can create different tables for different prompt types, gemini model etc, if you rename the table
cursor.execute(f'''
CREATE TABLE IF NOT EXISTS {table_name} (
               id INTEGER PRIMARY KEY AUTOINCREMENT,
               Firm_Name TEXT NOT NULL,
               Registered_Address TEXT,
               CEO TEXT,
               Establishment_Year INT,
               Number_Of_Employees INT,
               Revenue_Size INT,
               Website TEXT,
               NAICS_Code INT,
               SIC_Code INT,
               Status TEXT,
               Dissolvement_Year INT,
               Company_Type TEXT,
               Previous_Names TEXT, 
               Alternative_Names TEXT, 
               Key_Executive_Personnel TEXT
               )
               ''')


web_search_table_name = "firms_web_search_results"
webscraping_table_name = "firms_web_search_website_scrapings"
conn_web_results = sqlite3.connect("firms_web_search_results.db")
conn_websites = sqlite3.connect("firms_web_search_website_scrapings.db")
cursor_websearch = conn_web_results.cursor()
cursor_websites = conn_websites.cursor()

In [26]:
# Check existing tables in the database
existing_tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
print(existing_tables)

[('sqlite_sequence',), ('firm_properties_gemini_without_grounding_local_dataset_v1',)]


In [ ]:
# cursor.execute(f"DELETE FROM {table_name} WHERE id > 2023 AND id < 2028")
# conn.commit()

In [8]:
# table_to_drop = "firm_properties_gemini_without_grounding_local_dataset_v1"
# cursor.execute(f"DROP TABLE IF EXISTS {table_to_drop} ")
# conn.commit()

Check how many entries are here

In [9]:
cursor.execute(f''' SELECT COUNT(*) FROM {table_name} ''').fetchall()

[(2026,)]

Check how many firms have their website scrapings

In [10]:
cursor_websites.execute(f''' SELECT COUNT(*) FROM {webscraping_table_name}''').fetchall()

[(2917,)]

Get list of firms we'll loop through from the webscraping dataset

In [11]:
cursor_websites.execute(f''' SELECT id, Firm_Name FROM {webscraping_table_name} ''')
firm_list = cursor_websites.fetchall()

In [12]:
max_data_length_allowed = 1000000 # 90th percentile  
start_idx = 2020
end_idx = -1

In [13]:
def process_field(field, firm_id, firm_name, updated_query, max_data_length_allowed):

    # Open new connections for each thread (SQLite connections are not thread-safe)
    conn_websearch_thread = sqlite3.connect('firms_web_search_results.db')
    cursor_websearch_thread = conn_websearch_thread.cursor()

    conn_websites_thread = sqlite3.connect('firms_web_search_website_scrapings.db')
    cursor_websites_thread = conn_websites_thread.cursor()

    # Get the web search results
    cursor_websearch_thread.execute(f"SELECT {field} FROM firms_web_search_results WHERE id = ? AND Firm_Name = ?", (firm_id, firm_name,))
    web_search_result = cursor_websearch_thread.fetchone()
    if web_search_result is not None and web_search_result[0]:
        web_search_result = json.loads(web_search_result[0])
    else:
        web_search_result = "No web search data available"

    # Get the website scraping results
    cursor_websites_thread.execute(f"SELECT {field} FROM firms_web_search_website_scrapings WHERE id = ? AND Firm_Name = ?", (firm_id, firm_name,))
    website_scraping_result = cursor_websites_thread.fetchone()
    if website_scraping_result is not None and website_scraping_result[0]:
        website_scraping_result = json.loads(website_scraping_result[0])
        # Check if scraped contents are too long
        if len(json.dumps(website_scraping_result)) > max_data_length_allowed:
            print(f"Scraped contents for {firm_name} - {field} is too long. Skipping.")
            website_scraping_result = "No website scraping data available"
    else:
        website_scraping_result = "No website scraping data available"

    data_to_pass_llm = {
        f"Results of searching the web for {firm_name}": web_search_result,
        f"Scraped contents of top websites for {firm_name}": website_scraping_result
    }

    # Generate the prompt
    prompt = form_prompt(updated_query, data_to_pass_llm)

    # Call the LLM to get the response
    response = model.generate_content(prompt).text

    print("Response for ", firm_id, firm_name, field, response)

    # Close the thread-specific connections
    conn_websearch_thread.close()
    conn_websites_thread.close()

    return field, response


# Main processing loop
for firm_data in firm_list[start_idx:end_idx]:
    firm_id = firm_data[0]
    firm_name = firm_data[1]
    print("\n ---- Debug now for ", firm_name, firm_id)

    # Check if firm already exists, insert row if it doesn't
    cursor.execute(f"SELECT id FROM {table_name} WHERE id = ? AND Firm_Name = ?", (firm_id, firm_name,))
    firm_row = cursor.fetchone()
    if firm_row is None:
        print("Inserting new firm: ", firm_name)
        cursor.execute(f"INSERT INTO {table_name} (id, Firm_Name) VALUES (?, ?)", (firm_id, firm_name))
        conn.commit()
    else:
        # Get the existing firm's id
        print("Found row for firm: ", firm_name, firm_id)

    # Update queries with firm name
    updated_queries = {key: value.format(firm_name=firm_name) for key, value in field_to_query.items()}

    # List to hold fields that need processing
    fields_to_process = []

    for field in fields:
        # Check if the field value in the prediction database is NULL
        cursor.execute(f"SELECT {field} FROM {table_name} WHERE id = ? AND Firm_Name = ? AND {field} IS NOT NULL", (firm_id, firm_name,))
        if cursor.fetchone() is not None:
            # print(f"Field '{field}' already has data for firm '{firm_name}', skipping.")
            continue
        else:
            fields_to_process.append(field)

    # If there are no fields to process, continue to the next firm
    if not fields_to_process:
        continue

    # Use ThreadPoolExecutor to process fields in parallel
    results = {}
    with ThreadPoolExecutor(max_workers=14) as executor:
        # Create a future for each field
        futures = {
            executor.submit(process_field, field, firm_id, firm_name, updated_queries[field], max_data_length_allowed): field
            for field in fields_to_process
        }

        for future in as_completed(futures):
            field = futures[future]
            try:
                field, response = future.result()
                if response is not None:
                    results[field] = response
            except Exception as e:
                print(f"Error in future for field {field}: {e}")
    
    time.sleep(1) # to avoid hitting the gemini quote per minute

    # Update the database sequentially
    for field, response in results.items():
        # print(f"Updating database for {firm_name} - {field}")
        cursor.execute(f"UPDATE {table_name} SET {field} = ? WHERE Firm_Name = ? AND id = ?", (response, firm_name, firm_id))
        conn.commit()



 ---- Debug now for  "D" CORNER STONE CAFE LLC 2021
Found row for firm:  "D" CORNER STONE CAFE LLC 2021

 ---- Debug now for  "D" ENTERPRISES, INC. 2022
Found row for firm:  "D" ENTERPRISES, INC. 2022

 ---- Debug now for  "DANCE PRODUCTIONS" L.L.C. 2023
Found row for firm:  "DANCE PRODUCTIONS" L.L.C. 2023

 ---- Debug now for  Equifax 2024
Found row for firm:  Equifax 2024
Scraped contents for Equifax - Alternative_Names is too long. Skipping.
Scraped contents for Equifax - Establishment_Year is too long. Skipping.
Response for  2024 Equifax Establishment_Year 1899

Response for  2024 Equifax Dissolvement_Year N/A

Response for  2024 Equifax SIC_Code 7323

Response for  2024 Equifax Alternative_Names Retail Credit Company (RCC)

Response for  2024 Equifax Previous_Names Retail Credit Company

Response for  2024 Equifax Company_Type Public

Response for  2024 Equifax Website https://www.equifax.com/

Response for  2024 Equifax NAICS_Code 561450

Response for  2024 Equifax Registered_A

In [14]:
# Close the main database connection when done
conn.close()